# Demo workflows

Below are demonstrated the use of this library as a Python package and from the command line interface. Note that one step demonstrated below is to save datasets you need to run the local examples.

In [ ]:
import os

if os.name == 'nt':
    os.environ['CONDA_DLL_SEARCH_MODIFICATION_ENABLE'] = '1'

In [ ]:
import xarray as xr
import ocean_data_gateway as odg
import cf_xarray
import pandas as pd
import extract_model as em
import ocean_model_skill_assessor as omsa
import numpy as np
from IPython import display
from glob import glob

## Definitions

These dictionaries are not required for every type of run, but are required for identifying variables between datasets and models, and for running QC. They can be complicated if desired, but do not have to be. Once they are written they do not need to be modified much. More information about these dictionaries is available in the [`ocean_data_gateway` docs](https://ocean-data-gateway.readthedocs.io/en/latest/details.html#Configure-custom-criteria-and-variable-definitions).

In [ ]:
criteria = {
    "salt": {
        "name": "sea_water_practical_salinity$",
    },
}

var_def = {
    "salt": {"units": "psu", "fail_span": [-10, 60], "suspect_span": [-1, 45]},
}


A more complicated example of `criteria` is shown here, as it can be brought it from a nonlocal source:

In [ ]:
url = 'https://gist.githubusercontent.com/kthyng/c3cc27de6b4449e1776ce79215d5e732/raw/af448937e4896535e36ef6522df8460e8f928cd6/my_custom_criteria.py'
criteria = odg.return_response(url)
criteria

## Python library

This package can be used as a Python package or from the command line. Here we demonstrate its use as a Python package.

### Nonlocal reader: stations

This demonstrates the case in which your data files for comparing with model output are not available locally, but you know their names and where to find them (in this case, at the IOOS ERDDAP server). More information about these inputs is available in the [API docs](https://ocean-model-skill-assessor.readthedocs.io/en/latest/api.html).

In [ ]:
# omsa.set_criteria(criteria)

model_url = 'https://thredds.cencoos.org/thredds/dodsC/CENCOOS_CA_ROMS_FCST.nc'

dataset_ids = ['edu_humboldt_tdp','bodega-bay-bml_wts']

search = omsa.run(
                  loc_model=model_url,
                  approach='stations',
                  criteria=criteria,
                  var_def=var_def,
                  xarray_kwargs={'chunks': {'time': 1, 'depth': 1}},
                  time_range=['2021-9-1','2021-9-8'], 
                  variables=['salt'],
                  readers=[odg.erddap],
                  erddap={
                      'known_server': 'ioos'
                  },
                  stations = dataset_ids,
                  figname_map='nonlocal_library.png',
                  figname_data_prefix='nonlocal_library_',
                  parallel=False
)

#### Save local data files

Here we save the data files to use for the local examples.

In [ ]:
search['edu_humboldt_tdp'].squeeze().to_dataframe().to_csv('edu_humboldt_tdp.csv')

In [ ]:
search['bodega-bay-bml_wts'].to_netcdf('bodega-bay-bml_wts.nc')

### Nonlocal reader: region

This demonstrates the case in which your data files for comparing with model output are not available locally, and you want to perform a search in time and space. By default this would search in the spatial bounding box of the model output, but here we instead input a smaller bounding box so as to limit the number of datasets found and used. For several of the datasets, the model output isn't available (must be determined to be on land). 

In [ ]:
omsa.set_criteria(criteria)

model_url = 'https://thredds.cencoos.org/thredds/dodsC/CENCOOS_CA_ROMS_FCST.nc'
bbox = [-124.5, 40, -123.5, 42]
search = omsa.run(
                  loc_model=model_url,
                  approach='region',
                  bbox=bbox,
                  criteria=criteria,
                  var_def=var_def,
                  xarray_kwargs={'chunks': {'time': 1, 'depth': 1}},
                  time_range=['2021-9-1','2021-9-8'], 
                  variables=['salt'],
                  readers=[odg.erddap],
                  erddap={
                      'known_server': 'ioos'
                  },
                  figname_map='nonlocal_library_region.png',
                  figname_data_prefix='nonlocal_library_region_'
)

### Local reader

This demonstrates the case in which your data files for comparing with model output are available locally.

In [ ]:
omsa.set_criteria(criteria)

model_url = 'https://thredds.cencoos.org/thredds/dodsC/CENCOOS_CA_ROMS_FCST.nc'
filenames = [
             'edu_humboldt_tdp.csv',
            'bodega-bay-bml_wts.nc',
            ]

skip_units = True

search = omsa.run(
                  loc_model=model_url,
                  approach='region',
                  criteria=criteria,
                  var_def=var_def,
                  skip_units=skip_units,
                  xarray_kwargs={'chunks': {'time': 1, 'depth': 1}},
                  time_range=['2021-9-1','2021-9-8'], 
                  variables=['salt'],
                  readers=[odg.local],
                  local={'filenames': filenames},
                  figname_map='local_library.png',
                  figname_data_prefix='local_library_'
)

## Command line interface

Here we demonstrate the use of the command line interface mode of the package. The config yaml file must be modified for the necessary inputs. These are the same examples as above, but accomplished via the command line interface instead of the Python package.

### Local reader

In [ ]:
config_file = 'config_local.yaml'

In [ ]:
!python ../ocean_model_skill_assessor/CLI.py $config_file

In [ ]:
display.Image("local_cli.png")

In [ ]:
display.Image('local_cli_edu_humboldt_tdp.csv_salt.png')

In [ ]:
display.Image('local_cli_bodega-bay-bml_wts.nc_salt.png')

### Nonlocal workflow: stations

In [ ]:
config_file = 'config_nonlocal.yaml'

In [ ]:
!python ../ocean_model_skill_assessor/CLI.py $config_file

In [ ]:
display.Image("nonlocal_cli.png")

In [ ]:
display.Image('nonlocal_cli_edu_humboldt_tdp_salt.png')

In [ ]:
display.Image('nonlocal_cli_bodega-bay-bml_wts_salt.png')

### Nonlocal reader: region

In [ ]:
config_file = 'config_nonlocal_region.yaml'

In [ ]:
!python ../ocean_model_skill_assessor/CLI.py $config_file

In [ ]:
display.Image("nonlocal_cli_region.png")